# Grab Batter Info

## Monday, November 7
## Sam LaFell

### Purpose:
This notebook is to evaluate batter information. We wnat to pull in various batter metrics.
- Metrics could include:
    - WAR
    - PA
    - Expected Batting Average
    - BABIP
    - Expected Slugging Pct
    - Etc Etc

In [2]:
import pybaseball as pb
import pandas as pd

# Set max cols
pd.set_option('display.max_columns', 350)
pd.set_option('display.max_rows', 100)

## Data Needed:
-  Player ID
- Plate Appearances
- expected Batting Average


## First Up: Player ID
### Start with One Team, One Game

In [1]:
# batting stats by year, regular season
from pybaseball.lahman import *
batting = batting()

In [3]:
batting.head(10)

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,0,0,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,0,0,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,5,0,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,2,2,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,3,0,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0
5,armstbo01,1871,1,FW1,NaN,12,49,9,11,2,1,0,5.0,0.0,1.0,0,1.0,NaN,NaN,NaN,NaN,0.0
6,barkeal01,1871,1,RC1,NaN,1,4,0,1,0,0,0,2.0,0.0,0.0,1,0.0,NaN,NaN,NaN,NaN,0.0
7,barnero01,1871,1,BS1,NaN,31,157,66,63,10,9,0,34.0,11.0,6.0,13,1.0,NaN,NaN,NaN,NaN,1.0
8,barrebi01,1871,1,FW1,NaN,1,5,1,1,1,0,0,1.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
9,barrofr01,1871,1,BS1,NaN,18,86,13,13,2,1,0,11.0,1.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0


In [4]:
# SCD2 type table with start and end dates for each player at each team
grouped = batting.groupby(['playerID', 'teamID'])['yearID'].agg([min, max]).reset_index()
grouped.rename(columns={'min':'start', 'max':'end'})

,playerID,teamID,start,end
0,aardsda01,ATL,2015,2015
1,aardsda01,BOS,2008,2008
2,aardsda01,CHA,2007,2007
3,aardsda01,CHN,2006,2006
4,aardsda01,NYA,2012,2012
...,...,...,...,...
48776,zuverge01,DET,1954,1955
48777,zwilldu01,CHA,1910,1910
48778,zwilldu01,CHF,1914,1915
48779,zwilldu01,CHN,1916,1916


In [5]:
from pybaseball import batting_stats
from datetime import datetime

game_date = '2018-05-13'
game_year = datetime.strptime(game_date, '%Y-%m-%d').year

# get all of this season's batting data so far
data = batting_stats(game_year, qual=250)

# Team we want
team = 'ATL'
atl_batters = data.loc[data['Team']==team]

In [6]:
# Get Team Batters
atl_batters_lookup = pd.DataFrame()
atl_batter_list = atl_batters['Name'].values
for batter in atl_batter_list:
    last_name = batter.split(' ')[1]
    first_name = batter.split(' ')[0]
    batter_lookup = pb.playerid_lookup(last_name, first_name)
    atl_batters_lookup = pd.concat([atl_batters_lookup, batter_lookup])

Gathering player lookup table. This may take a moment.


In [7]:
# No team information in statcast data, can't limit to one team using this method, need the above batting_stats call to grab team info
from pybaseball import statcast_batter
atl_batters_playerids = atl_batters_lookup['key_mlbam'].values
statcast_batter(game_date, game_date, atl_batters_playerids[0])

Gathering Player Data


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,SI,2018-05-13,95.9,-1.91,6.04,"Freeman, Freddie",518692,570632,field_out,hit_into_play,NaN,NaN,NaN,NaN,4,"Freddie Freeman grounds out, shortstop Yadiel ...",R,L,R,MIA,ATL,X,6.0,ground_ball,1,2,2018,-1.18,0.74,-0.82,2.27,NaN,660670.0,NaN,0,6,Top,142.83,147.65,NaN,NaN,592663,NaN,NaN,5.547648,-139.511436,-6.010186,-16.584430,32.226430,-21.407928,3.54,1.60,14.0,89.1,-8.0,95.7,2025,6.3,530002,570632,592663,500743,516770,445988,592680,457727,621446,605119,54.23,0.200,0.191,0.0,1.0,0.0,0.0,2.0,40,4,Sinker,0,1,1,0,1,0,1,0,Infield shift,Standard,238,0.007,-0.143
1,SL,2018-05-13,86.1,-2.10,6.03,"Freeman, Freddie",518692,570632,NaN,called_strike,NaN,NaN,NaN,NaN,7,"Freddie Freeman grounds out, shortstop Yadiel ...",R,L,R,MIA,ATL,S,NaN,NaN,1,1,2018,0.03,0.87,-0.32,1.58,NaN,660670.0,NaN,0,6,Top,NaN,NaN,NaN,NaN,592663,NaN,NaN,4.112260,-125.276668,-6.041599,-0.480732,24.489871,-22.145530,3.69,1.68,NaN,NaN,NaN,85.6,2242,5.9,530002,570632,592663,500743,516770,445988,592680,457727,621446,605119,54.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,3,Slider,0,1,1,0,1,0,1,0,Infield shift,Standard,178,0.000,-0.079
2,SI,2018-05-13,93.7,-2.09,6.02,"Freeman, Freddie",518692,570632,NaN,foul,NaN,NaN,NaN,NaN,5,"Freddie Freeman grounds out, shortstop Yadiel ...",R,L,R,MIA,ATL,S,NaN,NaN,1,0,2018,-1.37,0.90,0.19,2.53,NaN,660670.0,NaN,0,6,Top,NaN,NaN,NaN,NaN,592663,NaN,NaN,8.740775,-136.043112,-5.138068,-18.565569,28.733516,-20.363898,3.54,1.60,NaN,NaN,NaN,92.8,1932,5.8,530002,570632,592663,500743,516770,445988,592680,457727,621446,605119,54.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,2,Sinker,0,1,1,0,1,0,1,0,Infield shift,Standard,237,0.000,-0.082
3,SI,2018-05-13,93.2,-2.14,6.07,"Freeman, Freddie",518692,570632,NaN,ball,NaN,NaN,NaN,NaN,11,"Freddie Freeman grounds out, shortstop Yadiel ...",R,L,R,MIA,ATL,B,NaN,NaN,0,0,2018,-1.37,0.78,-1.28,3.21,NaN,660670.0,NaN,0,6,Top,NaN,NaN,NaN,NaN,592663,NaN,NaN,5.050969,-135.529980,-3.194410,-17.536910,29.239857,-22.356911,3.76,1.79,NaN,NaN,NaN,91.9,1926,5.5,530002,570632,592663,500743,516770,445988,592680,457727,621446,605119,54.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,1,Sinker,0,1,1,0,1,0,1,0,Infield shift,Standard,240,0.000,0.045
4,SI,2018-05-13,94.3,-1.77,5.91,"Freeman, Freddie",518692,570632,walk,ball,NaN,NaN,NaN,NaN,13,Freddie Freeman walks. Ronald Acuna to 2nd.,R,L,R,MIA,ATL,B,NaN,NaN,3,0,2018,-1.29,0.94,-0.28,1.23,NaN,NaN,660670.0,2,3,Top,NaN,NaN,NaN,NaN,592663,NaN,NaN,6.702716,-136.968794,-8.458077,-17.659943,30.727316,-18.802751,3.74,1.85,NaN,NaN,NaN,93.9,2070,6.2,530002,570632,592663,500743,516770,445988,592680,457727,621446,605119,54.32,NaN,NaN,0.7,1.0,0.0,0.0,NaN,19,4,Sinker,0,1,1,0,1,0,1,0,Infield shift,Standard,234,-0.019,0.073
5,SL,2018-05-13,89.2,-1.90,5.96,"Freeman, Freddie",518692,570632,NaN,ball,NaN,NaN,NaN,NaN,11,Freddie Freeman walks. Ronald Acuna to 2nd.,R,L,R,MIA,ATL,B,NaN,NaN,2,0,2018,-0.79,1.34,-1.75,2.97,NaN,NaN,660670.0,2,3,Top,NaN,NaN,NaN,NaN,592663,NaN,NaN,2.025415,-129.737625,-4.041573,-9.230952,31.362924

In [8]:
atl_batters_lookup['key_mlbam'].values

array([518692, 645277, 542255, 622666, 455976, 452095, 621020, 518586,
       435559])

## Turn these into functions

1. First step, we want to identify a specific game

In [15]:
statcast()

start_dt 2023-04-15
end_dt 2023-04-16
This is a large query, it may take a moment to complete


100%|██████████| 2/2 [00:10<00:00,  5.46s/it]


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
3710,FC,2023-04-15,94.2,-1.76,6.44,"Jansen, Kenley",570482,445276,field_out,hit_into_play,<NA>,<NA>,<NA>,<NA>,7,Gio Urshela pops out to shortstop Yu Chang.,R,R,R,BOS,LAA,X,6,popup,0,2,2023,0.41,1.32,-0.39,1.87,<NA>,<NA>,<NA>,2,9,Top,118.27,159.04,<NA>,<NA>,624512,<NA>,<NA>,2.66631,-136.988185,-9.188643,4.621762,27.424848,-13.773974,3.43,1.69,109,74.0,67,95.1,2582,6.7,718576,445276,624512,671213,571771,646240,644374,608701,606132,657077,53.84,0.001,0.0,0.0,1,0,0,3,82,3,Cutter,9,7,7,9,7,9,7,9,Strategic,Standard,183,0.011,-0.052
3869,FC,2023-04-15,94.0,-1.91,6.37,"Jansen, Kenley",570482,445276,NaN,swinging_strike,<NA>,<NA>,<NA>,<NA>,12,Gio Urshela pops out to shortstop Yu Chang.,R,R,R,BOS,LAA,S,<NA>,NaN,0,1,2023,0.52,1.3,1.55,3.04,<NA>,<NA>,<NA>,2,9,Top,<NA>,<NA>,<NA>,<NA>,624512,<NA>,<NA>,7.85922,-136.579349,-5.774971,4.515186,33.05185,-14.757889,3.43,1.69,<NA>,<NA>,<NA>,94.0,2678,6.6,718576,445276,624512,671213,571771,646240,644374,608701,606132,657077,53.91,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,82,2,Cutter,9,7,7,9,7,9,7,9,Strategic,Standard,173,0.0,-0.027
4102,FC,2023-04-15,94.4,-2.04,6.29,"Jansen, Kenley",570482,445276,NaN,called_strike,<NA>,<NA>,<NA>,<NA>,7,Gio Urshela pops out to shortstop Yu Chang.,R,R,R,BOS,LAA,S,<NA>,NaN,0,0,2023,0.31,1.61,-0.65,2.22,<NA>,<NA>,<NA>,2,9,Top,<NA>,<NA>,<NA>,<NA>,624512,<NA>,<NA>,2.949878,-137.13804,-8.520352,3.230391,32.429587,-10.216366,3.43,1.69,<NA>,<NA>,<NA>,95.2,2587,7.0,718576,445276,624512,671213,571771,646240,644374,608701,606132,657077,53.5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,82,1,Cutter,9,7,7,9,7,9,7,9,Strategic,Standard,179,0.0,-0.022
4161,FC,2023-04-15,93.3,-2.03,6.33,"Jansen, Kenley",592669,445276,field_out,hit_into_play,<NA>,<NA>,<NA>,<NA>,4,Hunter Renfroe flies out to right fielder Alex...,R,R,R,BOS,LAA,X,9,fly_ball,0,0,2023,0.32,1.26,-0.32,2.33,<NA>,<NA>,<NA>,1,9,Top,171.57,105.81,<NA>,<NA>,624512,<NA>,<NA>,3.713762,-135.773184,-7.35461,3.204232,29.610359,-15.168431,3.5,1.73,259,92.9,46,94.1,2554,6.7,718576,445276,624512,671213,571771,646240,644374,608701,606132,657077,53.78,0.001,0.0,0.0,1,0,0,3,81,1,Cutter,9,7,7,9,7,9,7,9,Standard,Standard,177,0.023,-0.177
4309,SI,2023-04-15,95.0,-1.63,6.41,"Jansen, Kenley",543685,445276,field_out,hit_into_play,<NA>,<NA>,<NA>,<NA>,5,Anthony Rendon lines out to first baseman Tris...,R,R,R,BOS,LAA,X,3,line_drive,0,2,2023,-0.92,1.56,0.04,2.65,<NA>,<NA>,<NA>,0,9,Top,146.69,164.12,<NA>,<NA>,624512,<NA>,<NA>,6.505697,-138.004634,-7.709235,-13.225514,31.924589,-10.689458,3.3,1.62,110,90.6,6,95.7,2183,6.9,718576,445276,624512,671213,571771,646240,644374,608701,606132,657077,53.62,0.474,0.44,0.0,1,0,0,4,80,3,Sinker,9,7,7,9,7,9,7,9,Strategic,Standard,193,0.037,-0.146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [48]:
from pybaseball import statcast
import pandas as pd

def gather_game_info(game_day: str, team: str) -> pd.DataFrame:
    """
    Get Statcast data for a given game and team.

    Args:
        game_day (str): Date of the game in 'YYYY-MM-DD' format.
        team (str): Three-letter abbreviation for the team.

    Returns:
        pd.DataFrame: A DataFrame containing Statcast data for the game and team.

    Raises:
        ValueError: If no data is found for the given game and team.
    """
    # Statcast Info
    data = statcast(start_dt=game_day, end_dt=game_day, team=team)

    # No game for the team on this date
    if data.shape[0] == 0:
        raise ValueError(f"No data found for game on {game_day} for team {team}")

    # Get new team
    team_list = data[['home_team', 'away_team']].drop_duplicates()
    new_team = [new_team[0] for new_team in team_list.values if team[0] != team]
    
    # Get Statcast info for the 2nd team
    data2 = statcast(start_dt=game_day, end_dt=game_day, team=new_team[0])

    return pd.concat([data, data2])

data = gather_game_info('2023-04-15', 'LAD')

In [37]:

original_team = 'CHC'
[team[0] for team in team_list if team[0] != original_team]

['LAD']

In [23]:
# * Batters
def identify_home_away_batters(full_game_df):
    home_batters = full_game_df.loc[full_game_df['inning_topbot'] == 'Bot'].batter.unique().tolist()
    away_batters = full_game_df.loc[full_game_df['inning_topbot'] == 'Top'].batter.unique().tolist()
    return home_batters, away_batters

In [24]:
identify_home_away_batters(gather_game_info('2023-04-15', 'CHC'))

This is a large query, it may take a moment to complete


100%|██████████| 1/1 [00:01<00:00,  1.12s/it]


([444482,
  518792,
  678246,
  681546,
  502110,
  571970,
  518692,
  605141,
  605131,
  500743],
 [])